#***Text Generation using LSTM***

In [1]:
import numpy
import sys
from nltk.tokenize import RegexpTokenizer
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint

In [8]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [13]:
file = open("frankenstein.txt").read()

In [15]:
def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(filtered)

In [16]:
processed_inputs = tokenize_words(file)

In [17]:
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [18]:
input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

Total number of characters: 268848
Total vocab: 43


In [19]:
seq_length = 150
x_data = []
y_data = []

In [20]:
# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

In [21]:
n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

Total Patterns: 268698


In [22]:
X = numpy.reshape(x_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [24]:
y = np_utils.to_categorical(y_data)

In [31]:
y.shape

(268698, 43)

In [25]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [27]:
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [28]:
model.fit(X, y, ep+ochs=1, batch_size=2000, callbacks=desired_callbacks)
model.save('TG-LSTM.h5')

135/135 [==============================] - 5879s 44s/step - loss: 3.0832

Epoch 00001: loss improved from inf to 2.98471, saving model to model_weights_saved.hdf5


In [29]:
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [32]:
num_to_char = dict((i, c) for i, c in enumerate(chars))

#**Text Generation**

In [39]:
start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
"  heart thanked guiding spirit conducting safety place hoped notwithstanding adversary gibe meet grapple weeks period procured sledge dogs thus travers "
